In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

import cv2
import os

import numpy as np


# **Data Augmentation for each image**

In [5]:
#Origin Datasets
dir_train ={
'basket' : '/content/drive/MyDrive/TRAIN/Basketball',
'football' : '/content/drive/MyDrive/TRAIN/Football',
'rowing' : '/content/drive/MyDrive/TRAIN/Rowing',
'tennis' : '/content/drive/MyDrive/TRAIN/Tennis',
'swimming' : '/content/drive/MyDrive/TRAIN/Swimming',
'yoga' : '/content/drive/MyDrive/TRAIN/Yoga'
}

dir_validation = {
'basket' : '/content/drive/MyDrive/VALIDATION/Basketball',
'football' : '/content/drive/MyDrive/VALIDATION/Football',
'rowing' : '/content/drive/MyDrive/VALIDATION/Rowing',
'tennis' : '/content/drive/MyDrive/VALIDATION/Tennis',
'swimming' : '/content/drive/MyDrive/VALIDATION/Swimming',
'yoga' : '/content/drive/MyDrive/VALIDATION/Yoga'
}

#Augmented Datasets
dir_train_aug ={
'basket' : '/content/drive/MyDrive/Train/Basketball',
'football' : '/content/drive/MyDrive/Train/Football',
'rowing' : '/content/drive/MyDrive/Train/Rowing',
'tennis' : '/content/drive/MyDrive/Train/Tennis',
'swimming' : '/content/drive/MyDrive/Train/Swimming',
'yoga' : '/content/drive/MyDrive/Train/Yoga'
}

dir_validation_aug = {
'basket' : '/content/drive/MyDrive/Validation/Basketball',
'football' : '/content/drive/MyDrive/Validation/Football',
'rowing' : '/content/drive/MyDrive/Validation/Rowing',
'tennis' : '/content/drive/MyDrive/Validation/Tennis',
'swimming' : '/content/drive/MyDrive/Validation/Swimming',
'yoga' : '/content/drive/MyDrive/Validation/Yoga'
}

sports = {
'basket'   : 0,
'football' : 1,
'rowing'   : 2,
'tennis'   : 3,
'swimming' : 4,
'yoga'     : 5
}

In [26]:
import os
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
   
datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest',
      brightness_range = (0.5, 1.5))
    

for sport, dir_train_ori, dir_train_aug, dir_valid_ori, dir_valid_aug in zip(dir_train.keys(),dir_train.values(),dir_train_aug.values(),dir_validation.values(),dir_validation_aug.values()):
  
  for image in os.listdir(dir_train_ori):
    
    img = load_img(dir_train_ori+'/'+image) 
    x = img_to_array(img)
    x = x.reshape((1, ) + x.shape) 
    
    i = 0
    for batch in datagen.flow(x, batch_size = 1, save_to_dir = dir_train_aug, save_prefix = sport, save_format ='png'):
        i += 1
        if i > 9:
            break

  
  
  for image in os.listdir(dir_valid_ori):

    img = load_img(dir_valid_ori+'/'+image)
    x = img_to_array(img)
    x = x.reshape((1, ) + x.shape) 
    
    i = 0
    for batch in datagen.flow(x, batch_size = 1, save_to_dir = dir_valid_aug, save_prefix = sport, save_format ='png'):
        i += 1
        if i > 9:
            break

In [30]:
for sport, dir_train_ori, dir_train_aug, dir_valid_ori, dir_valid_aug in zip(dir_train.keys(), dir_train.values(), dir_train_aug.values(), dir_validation.values(), dir_validation_aug.values()):
  print(f"sport: {sport}\n train_orig: {len(os.listdir(dir_train_ori))}\n train_aug: {len(os.listdir(dir_train_aug))}\n validation_orig: {len(os.listdir(dir_valid_ori))}\n validation_aug: {len(os.listdir(dir_valid_aug))}")
  print("********************************************************************")

sport: basket
 train_orig: 270
 train_aug: 2377
 validation_orig: 76
 validation_aug: 734
********************************************************************
sport: football
 train_orig: 293
 train_aug: 2543
 validation_orig: 76
 validation_aug: 741
********************************************************************
sport: rowing
 train_orig: 264
 train_aug: 2338
 validation_orig: 73
 validation_aug: 708
********************************************************************
sport: tennis
 train_orig: 241
 train_aug: 2353
 validation_orig: 69
 validation_aug: 652
********************************************************************
sport: swimming
 train_orig: 264
 train_aug: 2321
 validation_orig: 75
 validation_aug: 730
********************************************************************
sport: yoga
 train_orig: 264
 train_aug: 2330
 validation_orig: 73
 validation_aug: 709
********************************************************************


# **Read data by `ImageDataGenerator`**

In [ ]:
TRAINING_DIR = r"/content/drive/MyDrive/Train"
VALIDATION_DIR = r"/content/drive/MyDrive/Validation"

train_datagen = ImageDataGenerator()
test_datagen  = ImageDataGenerator()


train_generator = train_datagen.flow_from_directory(
  TRAINING_DIR,
	target_size=(224,224),
	class_mode='categorical',
  batch_size=126 
  )   

validation_generator = test_datagen.flow_from_directory(	
  VALIDATION_DIR,
	target_size=(224,224),
	class_mode='categorical',
  batch_size=126
  )

# **Convert data to Numpy**

In [4]:
labels = ['Basketball', 'Football', 'Rowing', 'Tennis', 'Swimming', 'Yoga']

img_size = 224

def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                c = [0,0,0,0,0,0]
                c[class_num] = 1
                data.append([resized_arr, c])
            except Exception as e:
                print(e)
    return np.array(data)

train = get_data('/content/drive/MyDrive/TRAIN')
val = get_data('/content/drive/MyDrive/VALIDATION')

<ipython-input-4-36386d7b9bc9>:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


In [5]:
x_train = []
y_train = []
x_val = []
y_val = []

for feature, label in train:
  x_train.append(feature)
  y_train.append(label)

for feature, label in val:
  x_val.append(feature)
  y_val.append(label)

# Normalize the data
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255

x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

In [ ]:
print(y_train)
print(y_val)

In [ ]:
# TRAINING_DIR = r"/content/drive/MyDrive/TRAIN"

# training_datagen = ImageDataGenerator(
#       rescale = 1./255,
# 	    rotation_range=40,
#       width_shift_range=0.2,
#       height_shift_range=0.2,
#       shear_range=0.2,
#       zoom_range=0.2,
#       horizontal_flip=True,
#       fill_mode='nearest')


# VALIDATION_DIR = r"/content/drive/MyDrive/VALIDATION"
# validation_datagen = ImageDataGenerator(rescale = 1./255)



# train_generator = training_datagen.flow_from_directory(
# 	TRAINING_DIR,
# 	target_size=(150,150),
# 	class_mode='categorical',
#   batch_size=126,
#   save_to_dir = '/content/sample_data/Train'
# )

# validation_generator = validation_datagen.flow_from_directory(
# 	VALIDATION_DIR,
# 	target_size=(150,150),
# 	class_mode='categorical',
#   batch_size=126,
#   save_to_dir = '/content/sample_data/validation'
# )